In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append("/home/files/vision_tree")
from utils.session_config import setup_gpus

In [3]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
setup_gpus(memory_fraction=0.5)

1 Physical GPUs, 1 Logical GPUs


In [5]:
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import Input

In [6]:
data_0 = np.load("/home/files/datasets/mnist/0.npy")
label_0 = np.zeros(len(data_0), np.int32)
data_5 = np.load("/home/files/datasets/mnist/5.npy")
label_5 = np.ones(len(data_5), np.int32)
data_7 = np.load("/home/files/datasets/mnist/7.npy")
label_7 = np.ones(len(data_7), np.int32)*2

train_x = tf.concat([data_0, data_5, data_7], axis=0)
train_x = tf.cast(train_x, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
train_y = tf.concat([label_0, label_5, label_7], axis=0)

print(train_x.shape)
print(train_y.shape)

x = tf.data.Dataset.from_tensor_slices(train_x)
y = tf.data.Dataset.from_tensor_slices(train_y)
ds = tf.data.Dataset.zip((x, y))

(17609, 28, 28, 1)
(17609,)


In [11]:
test_set = ds.take(100).batch(20)

In [12]:
#test1
from utils.BEP import Back_etching_propagation
trained_model = tf.keras.models.load_model("/home/files/vision_tree/train/ckpt/mnist_cnn/057.h5")
bep_algorithm = Back_etching_propagation(trained_model)
#etching start
#bep_algorithm.categorical_etching(ds, num_class=3, save_path='057')

test1 = bep_algorithm.model

idx = 12
model = test1
for layer in model.layers:
    layer.trainable = False
model.layers[idx].trainable = True #turn on the target etching layer.
y = Input(shape=(), dtype=tf.int32)
i = model.layers[idx].output
_,o = model.layers[idx+1](i)

etching_model = tf.keras.Model(inputs=[model.input,y], outputs=o)

2 layer name :  conv2d_3
Conv2D
0 layer name :  re_lu_3
0 layer name :  max_pooling2d_2
2 layer name :  conv2d_4
Conv2D
0 layer name :  re_lu_4
0 layer name :  max_pooling2d_3
2 layer name :  conv2d_5
Conv2D
0 layer name :  re_lu_5
0 layer name :  global_average_pooling2d_1
2 layer name :  dense_1
Dense


In [29]:
etching_model.summary()

Model: "functional_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_8 (Te [(None, 28, 28, 1, 1 0           input_13[0][0]                   
__________________________________________________________________________________________________
etching__layer_8 (Etching_Layer (None, 28, 28, 1, 16 16          tf_op_layer_strided_slice_8[0][0]
__________________________________________________________________________________________________
functional_23 (Functional)      [(None, 26, 26, 16), 160         etching__layer_8[0][0]           
______________________________________________________________________________________

In [28]:
for L,idx in enumerate(bep_algorithm.etching_layer_indice[::-1]):
    print(idx)

17
12
7
2


In [34]:
etching_model.layers[12]

In [20]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1E-3)
for x,y in test_set:
    with tf.GradientTape() as tape:
        pred = etching_model([x,y], training=True)
        loss = tf.reduce_mean(pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [25]:
for L,idx in enumerate(bep_algorithm.etching_layer_indice[::-1]):
    print(model.layers[idx].get_weights())

[array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)]
[array([[1.0000094 , 0.99973005, 0.99949145, 1.0002098 , 0.9996077 ,
        1.        , 0.9999939 , 0.9995063 , 1.0001489 , 0.99933636,
        1.0000232 , 0.99992126, 0.9996089 , 0.9993713 , 0.99991655,
        0.9999421 ],
       [0.9999788 , 0.99995524, 0.99995714, 1.0000026 , 1.0000601 ,
        1.        , 1.0000714 , 0.9999181 , 1.0000172 , 0.999977  ,
        0.9999847 , 0.9999682 , 0.9999601 , 0.9999933 , 0.9999649 ,
        0.9999798 ],
       [0.9997662 , 0.9996923 , 0.9999578 , 0.9996152 , 1.0000083 ,
        1.        , 1.0001248 , 0.9997399 , 0.999995  , 1.0000896 ,
        0.999904  , 0.99979126, 0.9999206 , 1.0001749 , 0.9

In [27]:
model.layers[12].get_weights()

[array([[1.0000094 , 0.99973005, 0.99949145, 1.0002098 , 0.9996077 ,
         1.        , 0.9999939 , 0.9995063 , 1.0001489 , 0.99933636,
         1.0000232 , 0.99992126, 0.9996089 , 0.9993713 , 0.99991655,
         0.9999421 ],
        [0.9999788 , 0.99995524, 0.99995714, 1.0000026 , 1.0000601 ,
         1.        , 1.0000714 , 0.9999181 , 1.0000172 , 0.999977  ,
         0.9999847 , 0.9999682 , 0.9999601 , 0.9999933 , 0.9999649 ,
         0.9999798 ],
        [0.9997662 , 0.9996923 , 0.9999578 , 0.9996152 , 1.0000083 ,
         1.        , 1.0001248 , 0.9997399 , 0.999995  , 1.0000896 ,
         0.999904  , 0.99979126, 0.9999206 , 1.0001749 , 0.99969035,
         1.0002458 ],
        [1.0000362 , 1.0000196 , 0.99984866, 1.0000182 , 0.9999394 ,
         1.        , 0.9999433 , 1.0001312 , 1.0000261 , 0.9998075 ,
         0.9998524 , 1.0000379 , 1.0000207 , 0.99986583, 1.0000876 ,
         0.9999215 ],
        [1.0001618 , 0.9999246 , 0.9998074 , 1.0001961 , 1.0000049 ,
         1.    

In [35]:
#test2
test2 = tf.keras.models.load_model("/home/files/vision_tree/train/ckpt/mnist_cnn/057.h5")
test2.summary()
activation_idx = [4, 7]






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
re_lu_3 (ReLU)               (None, 26, 26, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 16)        2320      
_________________________________________________________________
re_lu_4 (ReLU)               (None, 11, 11, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 16)         

In [ ]:
etching_model = tf.keras.Model(inputs=[model.input,y], outputs=o)

In [ ]:
        for L,idx in enumerate(self.etching_layer_indice[::-1]):
            print(idx)
            for layer in self.model.layers:
                layer.trainable = False
            self.model.layers[idx].trainable = True #turn on the target etching layer.
            y = Input(shape=(), dtype=tf.int32)
            i = self.model.layers[idx].output
            _,o = self.model.layers[idx+1](i)
            if L == 0: #classification feature
                o = o*tf.one_hot(y, depth=num_class)
                etching_model = tf.keras.Model(inputs=[self.model.input,y], outputs=o)
            else:
                etching_model = tf.keras.Model(inputs=[self.model.input,y], outputs=o)
            optimizer = tf.keras.optimizers.SGD(learning_rate=1E-3)
            for x,y in ds:
                with tf.GradientTape() as tape:
                    pred = etching_model([x,y], training=True)
                    loss = tf.reduce_mean(pred)
                gradients = tape.gradient(loss, self.model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
            etching_weights = self.model.layers[idx].get_weights()[0]
            np.save("{}/{}.npy".format(save_path, len(self.etching_layer_indice)-L-1), etching_weights)